In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import psycopg2 

sys.path.append('../')

from sqlalchemy import create_engine

from db_utils import create_table, populate_table, insert_into_table
from pbp_utils import (get_qb_pass, get_game_results, get_receiving, get_rushing,
                       get_team_pass_yds, get_team_rush_yds, get_opp_pass,
                       get_def_stats)

In [ ]:
# loading up the postgres credentials in a separate file. we could use environment variables
# but doing it this way to mix it up!

with open('../env.txt', 'r') as file:
    env = file.read().splitlines()
    user = env[0]
    password = env[1]
    host = env[2]
    database = env[3]
    port = env[4]
    

URI = f'postgresql://{user}:{password}@{host}:{port}/{database}'

In [ ]:
# gettin the directory path
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
nfl_pbp_dir = 'data/pbp'
nfl_depth_chart_dir = 'data/depth_charts'

# loading all of the parquets files at once using the directory path
# replacing backslashes with empty strings to avoid csv errors
df = pd.read_parquet(f'{parent_dir}/{nfl_pbp_dir}')
df['desc'] = df['desc'].str.replace('\\', '', regex=True)
df['season'] = [int(x.split('_')[0]) for x in df.game_id]

print("the shape of the pbp df is:", df.shape)
print("the memory usage of the pbp df is :", df.memory_usage(deep=True).sum() / 1024**2, "MB")

# dropping columns to reduce the memory required in the hopes of populating the db
# all at once, but it didn't work. I'll have to do it in chunks.
unneeded_cols = ['nfl_api_id', 'old_game_id', 'home_coach', 'away_coach', 'game_stadium', 'weather']
df.drop(columns=[x for x in df.columns if 'lateral' in x], inplace=True)
df.drop(columns=[x for x in df.columns if 'player_2' in x], inplace=True)
df.drop(columns=[x for x in df.columns if 'total' in x], inplace=True)
df.drop(columns=unneeded_cols, inplace=True)

print("the memory usage of the pbp after dropping is :", df.memory_usage(deep=True).sum() / 1024**2, "MB")

In [ ]:
# creating the pbp table
create_table(df=df, table_name='pbp', URI=URI)

# creating a subset of the pbp df to test the populate_table function
testdf = df.iloc[:10000]
populate_table(df=testdf, table_name='pbp', URI=URI)

In [ ]:
# using the insert function to get the pbp  into the database
# quickly - estimated time with 50k chunksize is 2.5 minutes
# the print statements keep track of the progress and 
# enable error identification

chunks = [x for x in range(10000, len(df), 50000)]

for i in range(len(chunks)):
    
    if i < (len(chunks) - 1):
        print(i)
        df_ = df.iloc[chunks[i] : chunks[i+1]].copy()
        print("chunk final index:", df_.index[-1])
        try:
            insert_into_table(df=df_, table_name='pbp', URI=URI)
        except:
            print("error in insert", i)
            print(df_.index[-1])
            continue
    else:
        print(i)
        df_ = df.iloc[chunks[i]:].copy()
        print("last chunk final index:", df_.index[-1])
        try:
            insert_into_table(df=df_, table_name='pbp', URI=URI)
        except:
            print("error for some other reason", i)

In [ ]:
depth_charts = pd.read_parquet(f'{parent_dir}/{nfl_depth_chart_dir}')
depth_charts['depth_position'] = depth_charts['depth_position'].str.replace("\n", '')

create_table(df=depth_charts, table_name='depth_charts', URI=URI)
populate_table(df=depth_charts, table_name='depth_charts', URI=URI)

In [ ]:
player_stats = pd.read_parquet(f'{parent_dir}/data/player_stats')

create_table(df=player_stats, table_name='player_stats', URI=URI)
populate_table(df=player_stats, table_name='player_stats', URI=URI)

In [ ]:
nextgen = pd.read_parquet(f'{parent_dir}/data/nextgen')

create_table(df=nextgen, table_name='nextgen', URI=URI)
populate_table(df=nextgen, table_name='nextgen', URI=URI)

In [ ]:
misc = pd.read_parquet(f'{parent_dir}/data/misc')

create_table(df=misc, table_name='misc', URI=URI)
populate_table(df=misc, table_name='misc', URI=URI)

In [ ]:
snaps = pd.read_parquet(f'{parent_dir}/data/snap_counts')

create_table(df=snaps, table_name='snaps', URI=URI)
populate_table(df=snaps, table_name='snaps', URI=URI)

In [ ]:
lee_sharpe = 'https://raw.githubusercontent.com/nflverse/nfldata/master/data/games.csv'

ls_cols = ['game_id', 'overtime', 'home_rest', 'away_rest', 'div_game','roof', 
           'surface', 'temp', 'wind', 'home_coach', 'away_coach', 'referee']

ls = pd.read_csv(lee_sharpe, usecols=ls_cols)
ls = ls.sort_values(by='game_id')

In [ ]:
create_table(df=ls, table_name='lee_sharpe', URI=URI)
populate_table(df=ls, table_name='lee_sharpe', URI=URI)

In [ ]:
df['year'] = pd.to_datetime(df['game_date']).dt.year
df['two_point_conv_result'] = (
    df['two_point_conv_result']
    .map(
        {'success' : 1,
         'failure' : 0,
         }
    )
    .fillna('None')
)

df['game_date'] = pd.to_datetime(df['game_date'])
df['spread_line'] = df['spread_line'] * -1
df['field_goal_result'] = np.where(df['field_goal_result'] == 'made', 1, 0)
df['time_between'] = df.groupby(['game_id'])['game_seconds_remaining'].transform(lambda x: x.sub(x.shift(-1)).fillna(0))
df['play_type'] = np.where(df['two_point_attempt'] > 0.5, 'two_point_att', df['play_type'])
df['air_yards_to_sticks'] = df['air_yards'].sub(df['ydstogo'])
df['season'] = [int(x.split('_')[0]) for x in df.game_id]
df['blocked_player_name'] = np.where(df['blocked_player_name'].notnull(), 1, 0)
df['fg_0_39'] = np.where(((df['play_type'] == 'field_goal') & (df['kick_distance'].between(0,39))), 1, 0)
df['fg_40_49'] = np.where(((df['play_type'] == 'field_goal') & (df['kick_distance'].between(40,49))), 1, 0)
df['fg_50_on'] = np.where(((df['play_type'] == 'field_goal') & (df['kick_distance'].between(50,100))), 1, 0)
df['extra_point_result'] = np.where(df['extra_point_result'] == 'good', 1, 0)

In [ ]:
success = []

for a, b, c in zip(df['down'], df['ydstogo'], df['yards_gained']):
  frac = 0.4 * b
  
  if b == 0.0:
    success.append(np.nan)
  
  elif ((a==1) | (a==2)):
    if c >= frac:
      success.append(1)
    else:
      success.append(0)
  elif ((a==3) | (a==4)):
    if c >= b:
      success.append(1)
    else:
      success.append(0)

df['success'] = success

In [ ]:
def get_opp_pass(df=df):

  opp_pass = (
      df[df['pass_attempt'] == 1]
      .groupby(['game_id', 'defteam'], as_index=False)['yards_gained']
      .sum()
      .rename(columns={'defteam' : 'team',
                      'yards_gained' : 'opp_pass_yds'})
  )

  return opp_pass

In [ ]:
def get_opp_rush(df=df):

  opp_rush = (
      df[df['rush_attempt'] == 1]
      .groupby(['game_id', 'defteam'], as_index=False)['yards_gained']
      .sum()
      .rename(columns={'defteam' : 'team',
                      'yards_gained' : 'opp_rush_yds'})
  )

  return opp_rush

In [ ]:
def get_def_stats(df=df):

  def_cols = ['interception', 'season', 'return_touchdown', 'fumble', 
              'sack', 'epa']

  def_stats = (
      df[~df['desc'].str.contains('Aborted')].copy()
      .groupby(['game_id', 'defteam'], as_index=False)
      .agg({
          'interception' : 'sum',
          'season' : lambda x: x.unique()[0],
          'return_touchdown' : 'sum',
          'fumble_lost' : 'sum',
          'sack' : 'sum',
          'safety' : 'sum',
          'blocked_player_name' : 'sum'
      })
      .rename(columns={
          'defteam' : 'team',
          'interception' : 'def_int',
          'return_touchdown' : 'def_td',
          'sack' : 'def_sack',
          'fumble' : 'def_fumble',
          'blocked_player_name' : 'kick_blocked'})
  )
  
  return def_stats

In [ ]:
kicker_df = get_kicker_stats()

In [ ]:
def create_ref_dict():

  names = games_df['referee'].unique()
  ref_dict = {}

  for i in range(len(names)-1):
    sim = process.extractOne(names[i], names[i+1:])
    
    if sim[1] > 85:
      ref_dict[names[i]] = sim[0]

  # del ref_dict['Jim Sprenger']
  ref_rev_dict = {y : x  for x, y in ref_dict.items()}

  return ref_rev_dict

In [ ]:
def get_rolling_qb_epa(df=df):

  qb_epa_dropback = df[df['qb_dropback'] == 1][['game_id', 'passer_player_name', 'epa', 'wp']].copy()
  qb_epa_db = qb_epa_dropback[qb_epa_dropback['passer_player_name'].notnull()].copy()
  qb_epa_db.replace('Jos.Allen', 'J.Allen', inplace=True)
  qb_epa_db['1000_rolling_mean'] = qb_epa_db.groupby('passer_player_name')['epa'].transform(lambda x: x.shift().rolling(1000, min_periods=200).mean())
  qb_epa_db['exp_career_mean'] = qb_epa_db.groupby('passer_player_name')['epa'].transform(lambda x: x.shift().expanding().mean())

  return qb_epa_db.reset_index(drop=True)

In [ ]:
# getting red zone statistics

rz_df = df[df['yardline_100'] <= 20].copy()

rz_rushing = get_rushing(df=rz_df)
rz_receiving = get_receiving(df=rz_df)
rz_qbs = get_qb_pass(df=rz_df)

rz_qbs = (
    rz_qbs
    .merge(rz_rushing.drop(columns='success'), how='left', on=['game_id', 'player', 'player_id', 'team', 'total_line', 'home_team', 'away_team'])
    .fillna(0)
)

In [ ]:
# creating tables for rushing, passing, and receiving. merging the rushing
# and passing dataframes because sometimes quarterbacks run!

rush_df = get_rushing()
rec_df = get_receiving()
qbs = get_qb_pass()

qb_df = qbs.merge(rush_df.drop(columns='success'), how='left', on=['game_id', 'player', 'player_id', 'team', 'total_line', 'home_team', 'away_team'])
qb_df.fillna(0, inplace=True)
qb_df['total_epa'] = qb_df['rush_epa'].add(qb_df['epa'])

print(qb_df.shape)
print(rec_df.shape)
print(rush_df.shape)

In [ ]:
team_pass_yds = get_team_pass_yds()
team_rush_yds = get_team_rush_yds()

opp_rush = get_opp_rush()
opp_pass = get_opp_pass()

def_stats = get_def_stats()
team_scores = get_team_scores()

In [ ]:
games_df = get_game_results()

In [ ]:
ref_rev_dict = create_ref_dict()
games_df['referee'] = games_df['referee'].map(ref_rev_dict).fillna(games_df['referee'])

In [ ]:
drive_stats = get_drive_stats()
drive_summary = drive_stats.groupby(['game_id', 'team'], as_index=False).agg({'poss_time' : 'sum',
                                                                              'rush_attempt' : 'sum',
                                                                              'pass_attempt' : 'sum',
                                                                              'total_plays' : 'sum'})

In [ ]:
games_df = games_df.merge(drive_summary, how='left', on=['game_id', 'team'])

In [ ]:
games_df['pass_per_attempt'] = games_df['pass_yards'].div(games_df['pass_attempt']).round(1)
games_df['perc_pass'] = games_df['pass_attempt'].div(games_df['total_plays']).round(3)
games_df['sec_per_play'] = games_df['poss_time'].div(games_df['total_plays']).round(1)

In [ ]:
adj_epa = get_team_adjusted_epa()

off_epa_game = (
    adj_epa
    .groupby(['game_id', 'posteam', 'play_type'], as_index=False)['epa']
    .agg(['sum'])
    .reset_index()
    .pivot_table(index=['game_id', 'posteam'],
                 columns='play_type',
                 values='sum')
    .reset_index()
    .rename(columns={
        'pass' : 'off_pass_epa',
        'run' : 'off_run_epa',
        'posteam' : 'team'
    })
    .round(2)
)

def_epa_game = (
    adj_epa
    .groupby(['game_id', 'defteam', 'play_type'], as_index=False)['epa']
    .agg(['sum'])
    .reset_index()
    .pivot_table(index=['game_id', 'defteam'],
                 columns='play_type',
                 values='sum')
    .reset_index()
    .rename(columns={
        'pass' : 'def_pass_epa',
        'run' : 'def_run_epa',
        'defteam' : 'team'
    })
    .round(2)
)

In [ ]:
date_threshold = '2021-10-16'
pbp_df = df[df['game_date'] >= date_threshold].copy()

for chunk in np.array_split(pbp_df, 3):
  df_ = pd.DataFrame(chunk)

  insert_into_table(df_, 'pbp', URI)